this is just for practise

In [ ]:
from db_connection import get_connection
from tabulate import tabulate

creating db students 

In [ ]:
sql = """
DROP TABLE IF EXISTS source_grades;

-- Source data: Student grades
CREATE TABLE source_grades (
    student_email VARCHAR(100),
    student_name VARCHAR(100),
    course_code VARCHAR(20),
    course_name VARCHAR(100),
    instructor_name VARCHAR(100),
    grade_date DATE,
    score INT
);

INSERT INTO source_grades VALUES
('alice@school.edu', 'Alice', 'MATH101', 'Calculus I', 'Prof. Smith', '2024-01-15', 85),
('bob@school.edu', 'Bob', 'MATH101', 'Calculus I', 'Prof. Smith', '2024-01-15', 92),
('alice@school.edu', 'Alice', 'CS101', 'Programming', 'Prof. Jones', '2024-01-20', 88);

"""


In [ ]:

with get_connection() as connection:
    with connection.cursor() as cursor:
        cursor.execute(sql)
        connection.commit()   # REQUIRED for CREATE / INSERT



### YOUR TASK:
- 1. Create dim_students (student_key, student_email, student_name)
- 2. Create dim_courses (course_key, course_code, course_name, instructor_name)
- 3. Create dim_date (date_key, full_date, month_name, year)
- 4. Create fact_grades (grade_id, student_key, course_key, date_key, score)
- 5. Load dimensions with unique values
- 6. Load fact table by JOINing to get surrogate keys
- 7. Query: Average score per student
- 8. Query: Average score per course

In [ ]:
sql = """
    select * from source_grades;
"""


In [ ]:
with get_connection() as connection:
    with connection.cursor() as cursor:
        cursor.execute(sql)
        rows = cursor.fetchall()
        headers = [desc[0] for desc in cursor.description] # type: ignore

print(tabulate(rows, headers=headers, tablefmt="psql"))

task 1 - create dim_students (student_key, student_email, student_name)

DATA TYPES BCZ I KEEP FORGETTING THEM 

| Use case                     | Data type to say |
| ---------------------------- | ---------------- |
| Whole numbers (IDs, counts)  | `INT`            |
| Money, prices                | `DECIMAL(p,s)`   |
| Text (names, emails, labels) | `VARCHAR(n)`     |
| Dates only                   | `DATE`           |
| Date + time                  | `TIMESTAMP`      |
| True / False                 | `BOOLEAN`        |


In [ ]:
sql = """
DROP TABLE IF EXISTS dim_student;

CREATE TABLE dim_student (
    student_key SERIAL PRIMARY KEY,
    student_email VARCHAR(255) UNIQUE,
    student_name VARCHAR(100)
);

INSERT INTO dim_student (student_email, student_name)
SELECT DISTINCT
    student_email,
    student_name
FROM source_grades;

"""


In [ ]:

with get_connection() as connection:
    with connection.cursor() as cursor:
        cursor.execute(sql)
        connection.commit()   # REQUIRED for CREATE / INSERT


In [ ]:
sql = """
    select * from dim_student;
"""


In [ ]:
with get_connection() as connection:
    with connection.cursor() as cursor:
        cursor.execute(sql)
        rows = cursor.fetchall()
        headers = [desc[0] for desc in cursor.description] # type: ignore

print(tabulate(rows, headers=headers, tablefmt="psql"))

- 2. Create dim_courses (course_key, course_code, course_name, instructor_name)

In [ ]:
sql = """
DROP TABLE IF EXISTS dim_courses;

CREATE TABLE dim_courses (
    course_key SERIAL PRIMARY KEY,
    course_code VARCHAR(255) UNIQUE,
    course_name VARCHAR(100),
    instructor_name VARCHAR(100)
);

INSERT INTO dim_courses (course_code, course_name,instructor_name)
SELECT DISTINCT
    course_code,
    course_name,
    instructor_name
FROM source_grades;

"""


In [ ]:

with get_connection() as connection:
    with connection.cursor() as cursor:
        cursor.execute(sql)
        connection.commit()   # REQUIRED for CREATE / INSERT


In [ ]:
sql = """
    select * from dim_courses;
"""


In [ ]:
with get_connection() as connection:
    with connection.cursor() as cursor:
        cursor.execute(sql)
        rows = cursor.fetchall()
        headers = [desc[0] for desc in cursor.description] # type: ignore

print(tabulate(rows, headers=headers, tablefmt="psql"))

- 3. Create dim_date (date_key, full_date, month_name, year)

In [ ]:
sql = """
DROP TABLE IF EXISTS dim_date;

CREATE TABLE dim_date (
    date_key SERIAL PRIMARY KEY,
    full_date DATE ,
    month_name VARCHAR(20),
    year INT
);

INSERT INTO dim_date (full_date, month_name,year)
SELECT DISTINCT
    grade_date,
    to_char(grade_date,'Month'),
    extract(YEAR from grade_date)
FROM source_grades;

"""


In [ ]:

with get_connection() as connection:
    with connection.cursor() as cursor:
        cursor.execute(sql)
        connection.commit()   # REQUIRED for CREATE / INSERT


In [ ]:
sql = """
    select * from dim_date;
"""


In [ ]:
with get_connection() as connection:
    with connection.cursor() as cursor:
        cursor.execute(sql)
        rows = cursor.fetchall()
        headers = [desc[0] for desc in cursor.description] # type: ignore

print(tabulate(rows, headers=headers, tablefmt="psql"))

- 4. Create fact_grades (grade_id, student_key, course_key, date_key, score)

In [ ]:
sql = """
DROP TABLE IF EXISTS fact_grade;

CREATE TABLE fact_grade (
    grade_id SERIAL PRIMARY KEY,
    student_key INT REFERENCES dim_student(student_key),
    course_key INT REFERENCES dim_courses(course_key),
    date_key INT REFERENCES dim_date(date_key),
    score INT
);

INSERT INTO fact_grade (student_key, course_key,date_key,score)
SELECT DISTINCT
    s.student_key,
    c.course_key,
    d.date_key,
    sg.score
FROM source_grades sg
JOIN dim_student s on sg.student_email = s.student_email
JOIN dim_courses c on sg.course_code = c.course_code
JOIN dim_date d    ON sg.grade_date = d.full_date;
"""


In [ ]:

with get_connection() as connection:
    with connection.cursor() as cursor:
        cursor.execute(sql)
        connection.commit()   # REQUIRED for CREATE / INSERT


In [ ]:
sql = """
    SELECT * FROM fact_grade;
"""


In [ ]:
with get_connection() as connection:
    with connection.cursor() as cursor:
        cursor.execute(sql)
        rows = cursor.fetchall()
        headers = [desc[0] for desc in cursor.description] # type: ignore

print(tabulate(rows, headers=headers, tablefmt="psql"))